In [ ]:
!pip install transformers datasets evaluate sacrebleu peft  git+https://github.com/huggingface/huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoTokenizer

model_name = "xlm-roberta-base"

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tjasad/Slovene_SuperGLUE_BoolQ")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [6]:
def preprocess_data(examples):
    passage_inputs = [f"passage : {x} " for x in examples["passage"]]
    question_inputs = [f"question : {x} " for x in examples["question"]]
    inputs = [passage_input + question_input for passage_input, question_input in zip(passage_inputs, question_inputs)]

    model_inputs = tokenizer(inputs, max_length=400, truncation=True)

    model_inputs["labels"] = examples["label"]
    model_inputs["labels"] = [int(label) for label in model_inputs["labels"]]

    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_data, remove_columns=['idx', 'passage', 'label', 'question'], batched=True)

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import AutoModelForSequenceClassification, set_seed

set_seed(42)

id2label = {0: "False", 1: "True"}
label2id = {"False": 0, "True": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)

model.config.use_cache = False

In [11]:
from transformers import TrainingArguments

new_model_name = "fine_tuned_boolq__XLMroberta"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [12]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [13]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
50,0.671000,0.540649,0.777778,0.680556
100,0.483800,0.643689,0.666667,0.666667
150,0.253100,1.109076,0.666667,0.666667
200,0.064600,1.566720,0.722222,0.707170
250,0.001600,2.328857,0.611111,0.625514
300,0.001000,2.669799,0.611111,0.625514
350,0.000500,2.976221,0.611111,0.625514
400,0.000500,2.974111,0.611111,0.625514


TrainOutput(global_step=400, training_loss=0.18451482699718327, metrics={'train_runtime': 112.5918, 'train_samples_per_second': 28.421, 'train_steps_per_second': 3.553, 'total_flos': 427619187168720.0, 'train_loss': 0.18451482699718327, 'epoch': 33.333333333333336})

In [14]:
trainer.evaluate()

{'eval_loss': 2.974111318588257,
 'eval_accuracy': 0.6111111111111112,
 'eval_f1': 0.6255144032921812,
 'eval_runtime': 0.2062,
 'eval_samples_per_second': 87.312,
 'eval_steps_per_second': 14.552,
 'epoch': 33.333333333333336}

In [ ]:
trainer.push_to_hub(new_model_name)

In [16]:
from transformers import pipeline

adapter_name = "tjasad/" + new_model_name

# Example
text="passage : Bankovec za 20 evrov – Zaenkrat obstaja le ena celotna serija evrskih bankovcev, vendar pa izhaja nova serija, ki bo podobna sedanji. Evropska centralna banka bo pravočasno naznanila, kdaj bodo bankovci iz prve serije izgubili status zakonitega plačilnega sredstva. question : Ali je bankovec za 20 evrov iz prve serije še vedno zakonito plačilno sredstvo?"

classifier = pipeline("sentiment-analysis", model=adapter_name)
classifier(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

[{'label': 'False', 'score': 0.9996935129165649}]

In [ ]:
# GPU RAM = 7.9 GB